# CHAPTER 10
# Data Aggregation and Group Operations
- Categorizing a dataset and applying a function to each group, whether an aggregation or transformation, is often a critical component of a data analysis workflow. 
- After loading, merging, and preparing a dataset, you may need to compute **group statistics** or possibly **pivot tables** for reporting or visualization purposes. 
- **pandas** provides a flexible ***groupby*** interface, enabling you to slice, dice, and summarize datasets in a natural way.
- Aggregation of **time series** data, a special use case of groupby, is referred to as **resampling**.
- This Chapter will focus on:

    - Split a pandas object into pieces using one or more keys (in the form of functions, arrays, or DataFrame column names)
    - Calculate group summary statistics, like count, mean, or standard deviation, or a user-defined function
    - Apply within-group transformations or other manipulations, like normalization, linear regression, rank, or subset selection
    - Compute pivot tables and cross-tabulations
    - Perform quantile analysis and other statistical group analyses

## GroupBy Mechanics
- **group** operations can be described by the **split-apply-combine** method:
    1. Data contained in a pandas object, whether a Series, DataFrame, is **split into groups** based on one or more **keys** that you provide. The splitting is performed on a particular axis of an object. For example, a DataFrame can be grouped on its rows (axis=0) or its columns (axis=1). 
    2. A **function is applied** to each group, producing a new value. 
    3. The results of all those function applications are **combined into a result object**. The form of the resulting object will usually depend on what’s being done to the data.
- Each **grouping key** can take many forms, and the keys do not have to be all of the same type:
    - A list or array of values that is the same length as the axis being grouped
    - A value indicating a column name in a DataFrame
    - A dict or Series giving a correspondence between the values on the axis being grouped and the group names
    - A function to be invoked on the axis index or the individual labels in the index

In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
# Create example DataFrame
df = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': np.random.randn(5),
    'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-1.125319,0.133552
1,a,two,-0.933832,2.012404
2,b,one,0.473875,0.969330
3,b,two,-0.723459,1.095823
4,a,one,-0.338679,-1.346179


In [3]:
# Create a GroupBy object that has all of the information needed to apply some operation to
# each of the groups
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
# To compute group means we can call the GroupBy’s mean method
grouped.mean()

# The data (a Series) has been aggregated according to the group key,
# producing a new Series that is now indexed by the unique values in the key1 column

key1
a   -0.799277
b   -0.124792
Name: data1, dtype: float64

In [5]:
# If we group the data using two keys, the resulting Series now has a hierarchical index 
# consisting of the unique pairs of keys observed
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.731999
      two    -0.933832
b     one     0.473875
      two    -0.723459
Name: data1, dtype: float64

In [6]:
# Frequently the grouping information is found in the same DataFrame as the data you
# want to work on. In that case, you can pass column names as the group keys
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.799277,0.266593
b,-0.124792,1.032576


- In the first case **df.groupby('key1').mean()** there is no **key2** column in the result. 
- Because **df['key2']** is not numeric data, it is said to be a nuisance column, which is therefore excluded from the result. 
- By default, all of the numeric columns are aggregated, though it is possible to filter down to a subset.

In [7]:
# A generally useful GroupBy method is size, which returns a Series containing group sizes
df.groupby(['key1', 'key2']).size()

# Take note that any missing values in a group key will be excluded from the result

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups
- The GroupBy object supports **iteration**, generating a sequence of 2-tuples containing the group name along with the chunk of data.

In [8]:
# Print name & group
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -1.125319  0.133552
1    a  two -0.933832  2.012404
4    a  one -0.338679 -1.346179
b
  key1 key2     data1     data2
2    b  one  0.473875  0.969330
3    b  two -0.723459  1.095823


In [9]:
# In the case of multiple keys, the first element in the tuple will be a tuple of key values
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -1.125319  0.133552
4    a  one -0.338679 -1.346179
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.933832  2.012404
('b', 'one')
  key1 key2     data1    data2
2    b  one  0.473875  0.96933
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.723459  1.095823


### Selecting a Column or Subset of Columns
- Indexing a **GroupBy object** created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation.
- Especially for large datasets, it may be desirable to aggregate only a few columns.

In [10]:
# Compute means for just the data2 column and get the result as a DataFrame
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.606313
     two   2.012404
b    one   0.969330
     two   1.095823

In [11]:
# The object returned by this indexing operation is a grouped DataFrame if a list or
# array is passed or a grouped Series if only a single column name is passed as a scalar
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [12]:
# Apply the mean operation
s_grouped.mean()

key1  key2
a     one    -0.606313
      two     2.012404
b     one     0.969330
      two     1.095823
Name: data2, dtype: float64

### Grouping with Dicts and Series

In [13]:
# Create example DataFrame
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-1.564830,-0.245552,-1.505931,-0.462736,0.179649
Steve,0.641320,-0.365699,1.347487,-0.256650,-1.689484
Wes,-0.240334,NaN,NaN,-1.108959,-0.680253
Jim,-1.419160,-0.514462,-0.577810,0.015049,0.253633
Travis,-0.581581,-0.294024,-0.125603,-0.608118,-1.595669


In [14]:
# Mapping for columns
mapping = {'a': 'red', 'b': 'red', 'c': 'blue','d': 'blue', 'e': 'red', 'f' : 'orange'}

# Unused groups are ok like 'f'

In [15]:
# To group the columns we can just pass the dict to the groupby function
by_column = people.groupby(mapping, axis=1).sum()
by_column

,blue,red
Joe,-1.968668,-1.630734
Steve,1.090837,-1.413863
Wes,-1.108959,-0.920587
Jim,-0.562760,-1.679988
Travis,-0.733721,-2.471274


### Grouping with Functions
- Using Python functions is a more generic way of defining a group mapping compared with a dict or Series. 
- Any function passed as a group key will be called once per index value, with the return values being used as the group names.

In [16]:
# Group by the length of the names
people.groupby(len).sum()

,a,b,c,d,e
3,-3.224324,-0.760014,-2.083741,-1.556645,-0.246971
5,0.641320,-0.365699,1.347487,-0.256650,-1.689484
6,-0.581581,-0.294024,-0.125603,-0.608118,-1.595669


### Grouping by Index Levels
- For hierarchically indexed datasets you can aggregate using one of the levels of an axis index.

In [17]:
# Create example DataFrame
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.201346  2.633084 -0.043772 -0.353780 -2.021990
1      1.113503  1.446040 -0.727372  1.919501  0.222815
2     -0.469707 -0.640105  0.463208 -0.168469  1.003844
3     -0.392741  1.464164  0.186861  0.871145  0.277158

In [18]:
# To group by level, pass the level number or name using the level keyword
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation
- Aggregations refer to any data transformation that produces scalar values from arrays.
- Many common aggregations, such as mean, count, min and sum have optimized implementations.
- You can use aggregations of your own devising and additionally call any method that is also defined on the grouped object.
- Custom aggregation functions are generally much slower than the optimized functions.

**TABLE**: Optimized groupby methods

| Argument                  | Description |
| :---                  |    :----    |
|count| Number of non-NA values in the group
|sum| Sum of non-NA values
|mean| Mean of non-NA values
|median| Arithmetic median of non-NA values
|std, var| Unbiased (n – 1 denominator) standard deviation and variance
|min, max| Minimum and maximum of non-NA values
|prod| Product of non-NA values
|first, last| First and last non-NA values

In [19]:
# Our example DataFrame
df

,key1,key2,data1,data2
0,a,one,-1.125319,0.133552
1,a,two,-0.933832,2.012404
2,b,one,0.473875,0.969330
3,b,two,-0.723459,1.095823
4,a,one,-0.338679,-1.346179


In [20]:
# Create a GroupBy object
grouped = df.groupby('key1')

# Apply the quantile function to the GroupBy object
grouped['data1'].quantile(0.9)

key1
a   -0.457709
b    0.354142
Name: data1, dtype: float64

- To use your own aggregation functions, pass any function that aggregates an array to the **aggregate or agg method**.

In [21]:
# Define a function that aggregates
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [22]:
# Apply the function the the GroupBy object
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.786640,3.358583
b,1.197335,0.126492


In [23]:
# Some methods like describe also work, even though they are not aggregations
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.799277  0.410219 -1.125319 -1.029576 -0.933832 -0.636255   
b      2.0 -0.124792  0.846643 -0.723459 -0.424126 -0.124792  0.174542   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.338679   3.0  0.266593  1.683239 -1.346179 -0.606313  0.133552   
b     0.473875   2.0  1.032576  0.089443  0.969330  1.000953  1.032576   

                          
           75%       max  
key1                      
a     1.072978  2.012404  
b     1.064200  1.095823

### Column-Wise and Multiple Function Application
- **Aggregating** a Series or all of the columns of a DataFrame is a matter of using **aggregate** with the desired function or calling a method like **mean** or **std**. 
- However, you may want to aggregate using a different function depending on the column, or multiple functions at once.
- For **descriptive statistics** like those in Table *Optimized groupby methods*, you can pass the name of the function as a string to the **aggregate** method.

In [24]:
# Load tipping dataset
tips = pd.read_csv('examples/tips.csv')

In [25]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [26]:
# Group the tips by day and smoker
grouped = tips.groupby(['day', 'smoker'])
grouped

In [27]:
# Calculate the mean tip percentage by day & smoker
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [28]:
# If you pass a list of functions or function names instead, you get back a DataFrame
# with column names taken from the functions
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [29]:
# You can pass a a list of (name, function) tuples to define the name of the columns
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [30]:
# With a DataFrame you have more options, as you can specify a list of functions to
# apply to all of the columns or different functions per column

# Define a list of functions
functions = ['count', 'mean', 'max']

# Apply these fuction to columns 'tip_pct' & 'total_bill'
result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [31]:
# To apply different functions to one or more of the columns pass a dict to agg 
# that contains a mapping of column names to any of the functions

grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 'size' : 'sum'})

# A DataFrame will have hierarchical columns only if multiple functions are applied to
# at least one column

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes
- In all of the examples up until now, the aggregated data comes back with an **index**, potentially hierarchical, composed from the unique group key combinations. 
- Since this isn’t always desirable, you can disable this behavior in most cases by passing **as_index=False** to groupby.
- It’s always possible to obtain the result in this format by calling **reset_index** on the result. Using the as_index=False method avoids some unnecessary computations.

In [32]:
# Mean values by day & smoker with no index
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Apply: General split-apply-combine

- Returning to the tipping dataset from before, suppose you wanted to select the top five tip_pct values by group. 
- First, write a function that selects the rows with the largest values in a particular column.

In [33]:
# Define a function to select top 5 tip_pct
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

# Use the fucntion on the tips DataFrame
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [34]:
# Group by smoker and call apply with the top function
tips.groupby('smoker').apply(top, n=3, column='total_bill')

total_bill    tip smoker  day    time  size   tip_pct
smoker                                                           
No     156       48.17   5.00     No  Sun  Dinner     6  0.103799
       59        48.27   6.73     No  Sat  Dinner     4  0.139424
       212       48.33   9.00     No  Sat  Dinner     4  0.186220
Yes    102       44.30   2.50    Yes  Sat  Dinner     3  0.056433
       182       45.35   3.50    Yes  Sun  Dinner     3  0.077178
       170       50.81  10.00    Yes  Sat  Dinner     3  0.196812

- The top function is called on each **row group** from the DataFrame, and then the results are glued together using **pandas.concat**, labeling the pieces with the group names. 
- The result therefore has a **hierarchical index** whose inner level contains index values from the original DataFrame.

### Suppressing the Group Keys
- In the preceding examples, you see that the resulting object has a **hierarchical index** formed from the group keys along with the indexes of each piece of the original object. 
- You can disable this by passing **group_keys=False** to groupby.

In [35]:
# Suppress the group keys
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis
- **pandas** has some tools, in particular **cut** and **qcut**, for slicing data up into buckets with bins of your choosing or by sample quantiles (Chapter 8).
- Combining these functions with groupby makes it convenient to perform bucket or quantile analysis on a dataset.
- The **Categorical object** returned by **cut** can be passed directly to **groupby**.

In [36]:
# A simple random dataset
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
frame.head()

,data1,data2
0,-0.872167,1.236353
1,1.192181,-0.171920
2,1.694296,-2.479644
3,1.716710,-0.602923
4,-0.779073,1.345521


In [37]:
# Equal-length bucket categorization
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-1.691, -0.16]
1     (-0.16, 1.371]
2     (1.371, 2.901]
3     (1.371, 2.901]
4    (-1.691, -0.16]
5    (-1.691, -0.16]
6    (-1.691, -0.16]
7     (-0.16, 1.371]
8     (-0.16, 1.371]
9     (-0.16, 1.371]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.227, -1.691] < (-1.691, -0.16] < (-0.16, 1.371] < (1.371, 2.901]]

In [38]:
# Compute a set of statistics with the function get_stats
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

In [39]:
# Create a GroupBy oject using the quartiles defined above
grouped = frame.data2.groupby(quartiles)
grouped

In [40]:
# Apply the fucntion get_stats
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.227, -1.691]",-2.161520,2.071524,54.0,-0.233889
"(-1.691, -0.16]",-3.098525,3.088777,388.0,-0.026752
"(-0.16, 1.371]",-2.496468,2.896392,470.0,0.016637
"(1.371, 2.901]",-2.479644,2.011214,88.0,-0.128843


In [41]:
# To compute equal-size buckets based on sample quantiles, use qcut
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.609721,3.088777,100.0,-0.115038
1,-3.098525,2.860326,100.0,-0.037207
2,-1.757105,2.915145,100.0,0.073265
3,-2.787392,2.434825,100.0,-0.022756
4,-2.560291,2.051590,100.0,-0.118534
5,-1.852632,2.091191,100.0,-0.004012
6,-2.346060,2.896392,100.0,-0.062647
7,-2.003205,1.938480,100.0,-0.005657
8,-2.496468,1.909587,100.0,0.156016


### Example: Filling Missing Values with Group-Specific Values
- When cleaning up missing data, in some cases you will replace data observations using **dropna**, but in others you may want to impute (fill in) the null (NA) values using a fixed value or some value derived from the data. 
- **fillna** is the right tool to use and you can for example fill in NA values with the mean.

In [42]:
# Create example Series
s = pd.Series(np.random.randn(6))

# Ass some NA values
s[::2] = np.nan
s

0         NaN
1    0.482738
2         NaN
3   -1.158087
4         NaN
5   -0.664151
dtype: float64

In [43]:
# Fill NA values with the mean
s.fillna(s.mean())

0   -0.446500
1    0.482738
2   -0.446500
3   -1.158087
4   -0.446500
5   -0.664151
dtype: float64

In [44]:
# List of states
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']

# List of cardinal directions
group_key = ['East'] * 4 + ['West'] * 4

# Convert to series
data = pd.Series(np.random.randn(8), index=states)

# Add some missing values
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

data

Ohio          0.780480
New York     -1.052604
Vermont            NaN
Florida      -1.175353
Oregon       -1.628825
Nevada             NaN
California    0.422813
Idaho              NaN
dtype: float64

In [45]:
# We can fill the NA values using the group means

fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.780480
New York     -1.052604
Vermont      -0.482492
Florida      -1.175353
Oregon       -1.628825
Nevada       -0.603006
California    0.422813
Idaho        -0.603006
dtype: float64

In [46]:
# Fill in with predefined values that vary by group

fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.780480
New York     -1.052604
Vermont       0.500000
Florida      -1.175353
Oregon       -1.628825
Nevada       -1.000000
California    0.422813
Idaho        -1.000000
dtype: float64

### Example: Group Weighted Average and Correlation
- Under the **split-apply-combine** paradigm of **groupby**, operations between columns in a DataFrame or two Series, such as a group weighted average, are possible.

In [47]:
# Example dataset containing group keys, values, and some weights
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.942792,0.785229
1,a,0.271821,0.503582
2,a,-0.589278,0.803148
3,a,0.392462,0.481487
4,b,-1.427080,0.493777
5,b,0.754531,0.315605
6,b,0.070176,0.297493
7,b,-0.253259,0.670900


In [48]:
# Group weighted average by category
grouped = df.groupby('category') 
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.230384
b   -0.346253
dtype: float64

In [49]:
# Finance containing end-of-day prices for a few stocks and the S&P 500 index from Yahoo!
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [50]:
# Check the dataset
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [51]:
# Function that computes the pairwise correlation of each column with the 'SPX' column
spx_corr = lambda x: x.corrwith(x['SPX'])

In [52]:
# Percent change on close_px using pct_change
rets = close_px.pct_change().dropna()
rets[:5]

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386


In [53]:
# Group these percent changes by year
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [ ]:
# You could also compute inter-column correlations
# Here we compute the annual correlation between Apple and Microsoft
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

### Example: Group-Wise Linear Regression
- You can use **groupby** to perform more complex group-wise statistical analysis, as long as the function returns a pandas object or scalar value. 
- For example, you can define the following regress function (using the statsmodels econometrics library), which executes an ordinary least squares (OLS) regression on each chunk of data.

In [54]:
# Define the regress function
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [55]:
# Run a yearly linear regression for AAPL on SPX
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## Pivot Tables and Cross-Tabulation
- A **pivot table** is a data summarization tool frequently found in spreadsheet programs and other data analysis software. 
- It aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns.
- DataFrame has a **pivot_table** method, and there is also a top-level **pandas.pivot_table function**. 
- In addition to providing a convenience interface to groupby, pivot_table can add **partial totals**, also known as margins.
- The default pivot_table aggregation type is **mean**.

In [56]:
# Compute a table of group means by smoker & day
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [57]:
# Aggregate only tip_pct and size and group by time
# With smoker in the table columns and day in the rows
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [58]:
# Include partial totals
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

# All values are means without taking into account any grouping

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [59]:
# To use a different aggregation function, pass it to aggfunc
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

**TABLE**: pivot_table options

| Argument                  | Description |
| :---                  |    :----    |
|values| Column name or names to aggregate; by default aggregates all numeric columns
|index| Column names or other group keys to group on the rows of the resulting pivot table
|columns| Column names or other group keys to group on the columns of the resulting pivot table
|aggfunc| Aggregation function or list of functions ('mean' by default); can be any function valid in a groupby context
|fill_value| Replace missing values in result table
|dropna| If True, do not include columns whose entries are all NA
|margins| Add row/column subtotals and grand total (False by default)

### Cross-Tabulations: Crosstab
- A **cross-tabulation** (or crosstab for short) is a special case of a pivot table that computes group frequencies.

In [60]:
# Read data
data = pd.read_csv('datasets/bitly_usagov/sample_nationality.txt')
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [61]:
# Summarize this data by nationality and handedness
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10
